In [1]:
from sympy import *

In [26]:
r = symbols('r', nonnegative=True)
phi = symbols('phi')

A, B, C = symbols('A B C')

theta = Rational(1, 2) * phi
q1 = Function('q_1')(r)
q2 = Function('q_2')(r)

display(q1, q2)

q_1(r)

q_2(r)

In [29]:
class MyMArray(MutableDenseNDimArray):
    
    def __mul__(self, other):
        try:
            return super().__mul__(other)
        except ValueError:
            if not isinstance(other, MyMArray):
                raise ValueError("Need MyMArray for contraction")
            
            rank_self = self.rank()
            prod = tensorproduct(self, other)
            contraction = tensorcontraction(prod, (rank_self - 1, rank_self))
            
            if type(contraction) == Array:
                return MyMArray(contraction)
            else:
                return contraction
    
    def __matmul__(self, other):
        try:
            return super().__mul__(other)
        except ValueError:
            if not isinstance(other, MyMArray):
                raise ValueError("Need MyMArray for tensor product")

            return MyMArray( tensorproduct(self, other) )
    
    def __pow__(self, other):
        try:
            return super().__mul__(other)
        except ValueError:
            if not isinstance(other, MyMArray):
                raise ValueError("Need MyMArray for double contraction")

            rank_self = self.rank()
            prod = tensorproduct(self, other)
            contract = tensorcontraction(prod, (rank_self - 1, rank_self))
            contraction = tensorcontraction(contract, (rank_self - 2, rank_self - 1))
            
            if type(contraction) == Array:
                return MyMArray(contraction)
            else:
                return contraction
        
    def ip(self, other):
        if not isinstance(other, MyMArray):
            raise ValueError("Need MyMArray for inner product")
        
        rank_self = self.rank()
        prod = tensorproduct(self, other)
        while rank_self > 0:
            prod = tensorcontraction(prod, (0, rank_self))
            rank_self -= 1
            
        return prod
    
def lap(M):
    return M.diff(r, 2) + (1/r) * M.diff(r) + (1/r**2) * M.diff(phi, 2)

In [31]:
n = MyMArray([cos(theta), sin(theta), 0])
m = MyMArray([sin(theta), -cos(theta), 0])
l = MyMArray([0, 0, 1])

In [33]:
Q_mat = (q1 * n@n + q2 * m@m - (q1 + q2) * l@l)
display(Q_mat)

[[q_1(r)*cos(phi/2)**2 + q_2(r)*sin(phi/2)**2, q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2), 0], [q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2), q_1(r)*sin(phi/2)**2 + q_2(r)*cos(phi/2)**2, 0], [0, 0, -q_1(r) - q_2(r)]]

In [34]:
lap_Q_mat = simplify(lap(Q_mat))
display(lap_Q_mat)

[[(-r**2*sin(phi/2)**2*Derivative(q_1(r), (r, 2)) + r**2*sin(phi/2)**2*Derivative(q_2(r), (r, 2)) + r**2*Derivative(q_1(r), (r, 2)) - r*sin(phi/2)**2*Derivative(q_1(r), r) + r*sin(phi/2)**2*Derivative(q_2(r), r) + r*Derivative(q_1(r), r) + q_1(r)*sin(phi/2)**2 - q_1(r)/2 - q_2(r)*sin(phi/2)**2 + q_2(r)/2)/r**2, (r**2*(Derivative(q_1(r), (r, 2)) - Derivative(q_2(r), (r, 2))) + r*(Derivative(q_1(r), r) - Derivative(q_2(r), r)) - q_1(r) + q_2(r))*sin(phi)/(2*r**2), 0], [(r**2*(Derivative(q_1(r), (r, 2)) - Derivative(q_2(r), (r, 2))) + r*(Derivative(q_1(r), r) - Derivative(q_2(r), r)) - q_1(r) + q_2(r))*sin(phi)/(2*r**2), (r**2*sin(phi/2)**2*Derivative(q_1(r), (r, 2)) - r**2*sin(phi/2)**2*Derivative(q_2(r), (r, 2)) + r**2*Derivative(q_2(r), (r, 2)) + r*sin(phi/2)**2*Derivative(q_1(r), r) - r*sin(phi/2)**2*Derivative(q_2(r), r) + r*Derivative(q_2(r), r) - q_1(r)*sin(phi/2)**2 + q_1(r)/2 + q_2(r)*sin(phi/2)**2 - q_2(r)/2)/r**2, 0], [0, 0, (r*(-Derivative(q_1(r), (r, 2)) - Derivative(q_2(r), 

In [36]:
dQ_dt = -A * Q_mat - B * Q_mat * Q_mat - C * (Q_mat**Q_mat) * Q_mat + lap(Q_mat)
# dQ_dt = simplify(dQ_dt)

display(dQ_dt)

[[-A*(q_1(r)*cos(phi/2)**2 + q_2(r)*sin(phi/2)**2) - B*(q_1(r)*cos(phi/2)**2 + q_2(r)*sin(phi/2)**2)**2 - B*(q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2))**2 - C*(q_1(r)*cos(phi/2)**2 + q_2(r)*sin(phi/2)**2)*((q_1(r)*sin(phi/2)**2 + q_2(r)*cos(phi/2)**2)**2 + (q_1(r)*cos(phi/2)**2 + q_2(r)*sin(phi/2)**2)**2 + 2*(q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2))**2 + (-q_1(r) - q_2(r))**2) + sin(phi/2)**2*Derivative(q_2(r), (r, 2)) + cos(phi/2)**2*Derivative(q_1(r), (r, 2)) + (sin(phi/2)**2*Derivative(q_2(r), r) + cos(phi/2)**2*Derivative(q_1(r), r))/r + (q_1(r)*sin(phi/2)**2 - q_1(r)*cos(phi/2)**2 - q_2(r)*sin(phi/2)**2 + q_2(r)*cos(phi/2)**2)/(2*r**2), -A*(q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2)) - B*(q_1(r)*sin(phi/2)**2 + q_2(r)*cos(phi/2)**2)*(q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2)) - B*(q_1(r)*cos(phi/2)**2 + q_2(r)*sin(phi/2)**2)*(q_1(r)*sin(phi/2)*cos(phi/2) - q_2(r)*sin(phi/2)*cos(phi/2)) - C*(q_1(r)*sin(phi/2)*

In [49]:
dQ_dt[0, 0].subs(phi, 0)

-A*q_1(r) - B*q_1(r)**2 - C*((-q_1(r) - q_2(r))**2 + q_1(r)**2 + q_2(r)**2)*q_1(r) + Derivative(q_1(r), (r, 2)) + Derivative(q_1(r), r)/r + (-q_1(r) + q_2(r))/(2*r**2)

In [47]:
dQ_dt[0, 0].subs(phi, pi)

-A*q_2(r) - B*q_2(r)**2 - C*((-q_1(r) - q_2(r))**2 + q_1(r)**2 + q_2(r)**2)*q_2(r) + Derivative(q_2(r), (r, 2)) + Derivative(q_2(r), r)/r + (q_1(r) - q_2(r))/(2*r**2)

## Solving for S_bulk

In [54]:
d_ij = MyMArray(eye(3))

S = symbols('S')
Q_uni = S * (n@n - Rational(1, 3) * d_ij)

display(Q_uni)

[[S*(cos(phi/2)**2 - 1/3), S*sin(phi/2)*cos(phi/2), 0], [S*sin(phi/2)*cos(phi/2), S*(sin(phi/2)**2 - 1/3), 0], [0, 0, -S/3]]

In [56]:
f_LdG = (Rational(1, 2) * A * Q_uni**Q_uni
         + Rational(1, 3) * B * Q_uni ** (Q_uni * Q_uni)
         + Rational(1, 4) * C * (Q_uni ** Q_uni)**2)
f_LdG = simplify(f_LdG)
display(f_LdG)

S**2*(108*A + 24*B*S + 36*C*S**2)/324

In [57]:
df_dS = simplify( f_LdG.diff(S) )
display(df_dS)

2*S*(3*A + B*S + 2*C*S**2)/9

In [60]:
sol = solve(df_dS / S, S)
display(sol[0], sol[1])

(-B - sqrt(-24*A*C + B**2))/(4*C)

(-B + sqrt(-24*A*C + B**2))/(4*C)

In [63]:
import math

In [68]:
def calc_S_bulk(A, B, C):
    return -B + math.sqrt(-24 * A * C + B**2) / C

## Solving bvp numerically

* First, need to rewrite as vector differential equation
* Will be 4D to make it first order
* Call $y = \left[q_1, q_2, q_1', q_2'\right]^T$
* Call $f = \begin{bmatrix}
                q_1' \\ 
                q_2' \\
                -\frac{1}{r} q_1' 
                + \frac{1}{2 r^2} (q_1 - q_2)
                + A q_1 + B q_1^2 + C \left( (q_1 + q_2)^2 + q_1^2 + q_2^2 \right)q_1 \\
                - \frac{1}{r} q_2'
                + \frac{1}{2 r^2} (q_2 - q_1)
                + A q_2 + B q_2^2 + C \left( (q_1 + q_2)^2 + q_1^2 + q_2^2 \right)q_2
             \end{bmatrix}$
* Then $\frac{dy}{dr} = f$

* More explicitly:
* $f = \begin{bmatrix}
            y_2 \\
            y_3 \\
            -\frac{1}{x} y_3 
            + \frac{1}{2x^2} (y_0 - y_1)
            + A y_0 + B y_0^2 + 2C (y_0^2 + y_1^2 + y_0 y_1) y_0 \\
            -\frac{1}{x} y_4 
            + \frac{1}{2x^2} (y_1 - y_0)
            + A y_1 + B y_1^2 + 2C (y_0^2 + y_1^2 + y_0 y_1) y_1
       \end{bmatrix}$

* Note: we want to solve this as an initial value problem, where we start at some far-away distance $x_0$ with values $q_1(x_0) = q_2(x_0) = S_\text{bulk}$, and $q_1'(x_0) = q_2'(x_0) = 0$.
* Thus we take $x \to x_0 - x$. Making this substitution we get:
* $f = \begin{bmatrix}
            y_2 \\
            y_3 \\
            -\frac{1}{x - x_0} y_3 
            + \frac{1}{2(x - x_0)^2} (y_0 - y_1)
            + A y_0 + B y_0^2 + 2C (y_0^2 + y_1^2 + y_0 y_1) y_0 \\
            -\frac{1}{x - x_0} y_4 
            + \frac{1}{2(x - x_0)^2} (y_1 - y_0)
            + A y_1 + B y_1^2 + 2C (y_0^2 + y_1^2 + y_0 y_1) y_1
       \end{bmatrix}$

In [80]:
def fun(x, y, A, B, C, x0):
    
    dy = np.zeros(y.shape)
    
    dy[0] = y[2]
    dy[1] = y[3]
    dy[2] = (-1/(x - x0) * y[2] 
                + 1 / (2 * (x - x0)**2) * (y[0] - y[1])
                + A * y[0] 
                + B * y[0]**2 
                + 2 * C * (y[0]**2 + y[1]**2 + y[0] * y[1]) * y[0])
    dy[3] = (-1/(x - x0) * y[3] 
                + 1 / (2 * (x - x0)**2) * (y[1] - y[0])
                + A * y[1] 
                + B * y[1]**2 
                + 2 * C * (y[0]**2 + y[1]**2 + y[0] * y[1]) * y[1])
    
    return dy

In [81]:
from scipy.integrate import ode
import numpy as np

In [84]:
A = -0.064
B = -1.57
C = 1.29

S_bulk = calc_S_bulk(A, B, C)

y0 = np.array([S_bulk, -0.5 * S_bulk, 0, 0])
x0 = 0

xf = 10
dx = 0.1

In [85]:
r = ode(fun)
r.set_initial_value(y0, x0).set_f_params(A, B, C, xf)

while r.successful() and r.t < xf:
    print(r.t + dx, r.integrate(r.t + dx))

0.1 [ 3.45260402 -1.78992757  5.18596972 -3.92874114]
0.2 [  4.35555175  -2.47685204  14.01087812 -10.69834232]
0.30000000000000004 [  6.80828678  -4.36191149  41.56375953 -32.09530701]
0.4 [  19.04418752  -13.87529147  357.92759826 -279.03856095]
0.5 [ 4.79369822e+11 -3.74334668e+11  2.37622890e+23 -1.85557124e+23]
